<a href="https://colab.research.google.com/github/vinbaskaran/AI_projects/blob/main/Student_ShopAssist_AI_2_0_Project_28thJune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ShopAssist AI 2.0

In this sample solution, we have provided a method to combine function calling API with the ShopAssist AI 2.0's system design to make it more seamless, and with less lines of code!

This solution, redefines our system design, and refines the overall conversation flow. Also, the output of the layers are now more consistent and therefore, require less manipulation for a seamless conversation.

Note that this is an outline solution, and hence contains primarily the function calling API prototype only. You are suggested to modify this further, and add additional layers (moderation, etc.) to modify the output as well

In [ ]:
#Install openai
!pip install openai==0.28

In [ ]:
## Import the necessary libraries for building the project
import os, json, ast
import openai
import pandas as pd

from google.colab import userdata

openai.api_key = userdata.get('api_key_s')

In [ ]:
## Function Description for the Function Calling API

function_descriptions = [
            {
                "name": "compare_laptops_with_user",
                "description": "Get the top 3 laptops from the catalogue, that best matches what the user is asking based on 'GPU intensity','Display quality','Portability','Multitasking','Processing speed' & 'Budget",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "gpu intensity": {
                            "type": "string",
                            "description": "The requirement of the user in GPU capacity classfied as low, medium or high" ,
                        },
                        "display quality": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's Display Quality & capacity classfied as low, medium or high" ,
                        },
                        "portability": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's portability classfied as low, medium or high" ,
                        },
                        "multitasking": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's Multitasking classfied as low, medium or high" ,
                        },
                        "processing speed": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's Processing speed classfied as low, medium or high" ,
                        },
                        "budget": {
                            "type": "integer",
                            "description": "The maximum budget of the user" ,
                        },

                    },
                    "required": ["GPU intensity","Display quality","Portability","Multitasking","Processing speed","Budget"],
                },
            }
        ]

In [ ]:
def initialize_conversation():
    delimiter = "####"

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You are trying to understand the user's requirement for these laptop's features: ('gpu intensity','display quality','portability','multitasking','processing speed','budget')
    You need to ask relevant questions and understand the user need for each feature by analysing the user's responses.
    After understanding their requirements, you'll use a function call to suggest the top 3 laptops with their respective user match score.
    Recommend these laptops and answer any user's query about them.

    {delimiter} Here are certain guidelines that you need to follow:
    Don't ask questions about more than 2 features at a time.
    If the user's budget is less than says less than 25000 INR, please mention that there are no laptops in that range.
    Recommend the top3 laptops in the following format:
    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    3. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    {delimiter}

    {delimiter}To find the top3 laptops, you need to have the following chain of thoughts:
    Thought 1: Ask one question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    Answer "Yes" or "No" to indicate if you understand the requirements. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n

    Thought 2: Now, you are trying to understand the requirements for other features which you couldn't in the previous step.
    Ask questions to strengthen your understanding of the user's profile.
    Don't ask questions about more than 2 features at a time.
    Answer "Yes" or "No" to indicate if you understood all the needs of the features and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the features whose needs you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the feature you want to understand the need for.{delimiter}
    {delimiter}

    {delimiter} Here is a sample conversation between the user and assistant:
    Assistant: "Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?"
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    {delimiter}


    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation

In [ ]:
def get_chat_model_completions(messages):
    response = openai.ChatCompletion.create(
        model = 'gpt-4o',
        messages = messages,
        temperature = 0,
        max_tokens = 500,

        functions= function_descriptions,
        function_call = "auto"

    )
    return response.choices[0].message

### Example Inut & Output

laptop_recommendation = '[{"Laptop Name": "Laptop A", "Price": 75000, "Score": 5},
                          {"Laptop Name": "Laptop B", "Price": 79000, "Score": 4},
                          {"Laptop Name": "Laptop C", "Price": 78000, "Score": 2}]'

recommendation_validation(laptop_recommendation)


{"gpu":"high", "portability":"medium"}

In [ ]:
import pandas as pd
import ast
import re

def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

def compare_laptops_with_user(user_req_string):
  budget = user_req_string.get('budget', '0')
  laptop_df= pd.read_csv('updated_laptop.csv')
  filtered_laptops = laptop_df.copy()
  filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
  filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()

  mappings = {
          'low': 0,
          'medium': 1,
          'high': 2
      }

  # Create 'Score' column in the DataFrame and initialize to 0
  filtered_laptops['Score'] = 0
  for index, row in filtered_laptops.iterrows():
      user_product_match_str = row['laptop_feature']
      laptop_values = extract_dictionary_from_string(user_product_match_str)
      score = 0

      for key, user_value in user_req_string.items():
        if key.lower() == 'budget':
            continue  # Skip budget comparison
        laptop_value = laptop_values.get(key, None)
        laptop_mapping = mappings.get(laptop_value.lower(), -1)
        user_mapping = mappings.get(user_value.lower(), -1)
        if laptop_mapping >= user_mapping:
          ### If the laptop value is greater than or equal to the user value the score is incremented by 1
          score += 1

      filtered_laptops.loc[index, 'Score'] = score

  # Sort the laptops by score in descending order and return the top 5 products
  top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
  top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

  return top_laptops.to_json(orient='records')

def recommendation_validation(laptop_recommendation):

    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
      if data[i]['Score'] > 2:
        data1.append(data[i])

    return json.dumps(data1)

In [ ]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()

    introduction = get_chat_model_completions(conversation)["content"]
    print(introduction + '\n')
    user_input = ""

    while(user_input != "exit"):
      user_input = input("")
      conversation.append({"role": "user", "content": user_input})


      # Step 1: Pass the user input to GPT
      response_assistant = get_chat_model_completions(conversation)

      if response_assistant.get("function_call"):
        print("\nThank you for providing all the information. Kindly wait, while I fetch the products\n")


        # Step 2: Extract top3 laptops by calling the external function
        function_name = response_assistant["function_call"]["name"]

        function_args = json.loads(response_assistant["function_call"]["arguments"])
        top_3_laptops = compare_laptops_with_user(function_args)
        function_response = recommendation_validation(top_3_laptops)


        if len(function_response) == 0:
          print("Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
          break

        # Step 3: send the info on the function call and function response to GPT
        conversation.append(response_assistant)
        conversation.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )
        recommendation = get_chat_model_completions(conversation)
        conversation.append({"role": "assistant", "content": recommendation["content"]})
        print("\n" +recommendation["content"] + "\n")
      else:
        conversation.append({"role": "assistant", "content": response_assistant["content"]})
        print("\n" +  response_assistant["content"] + "\n")


In [ ]:
dialogue_mgmt_system()

Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements or let me know what you'll primarily use the laptop for?



KeyboardInterrupt: Interrupted by user

I am looking for a gaming laptop under 80,000 with an high end GPU and meddium diplay configuration